### Railways FRA Data Analysis 
#### (DS 6001 - Final Pipeline)

In [1]:
import sqlite3
import pandas as pd
import numpy as np
%matplotlib inline

In [2]:
conn = sqlite3.connect("fra.db")

In [3]:
df = pd.read_sql('SELECT * FROM inc', conn, index_col='index')

In [4]:
new_data = pd.read_csv('fra_newdata.csv', index_col=False, header = 0)

In [5]:
max_year = max(df.YEAR4)
max_year

2018

In [6]:
max_month = max(df['MONTH'].loc[df['YEAR4'] == max_year])
max_month

7

In [7]:
df2 = df.loc[df['YEAR4'] == max_year]
max_day = max(df2['DAY'].loc[df2['MONTH'] == max_month])
max_day

31

Will subset data 1 August 2018 onwards.

In [8]:
df2 = new_data.loc[new_data['YEAR4'] >= max_year]
df2 = df2[df2['MONTH'] > max_month]
df2.head()

,IYR,IMO,RAILROAD,INCDTNO,IYR2,IMO2,RR2,INCDTNO2,IYR3,IMO3,...,NARR15,RCL,Latitude,Longitud,SIGNAL,MOPERA,ADJUNCT1,ADJUNCT2,ADJUNCT3,SUBDIV
21,18,8,KCS,18082502,NaN,NaN,NaN,NaN,18,8,...,NaN,0.0,32.541800,-93.844812,1,5,Z,NaN,NaN,SHREVEPORT TERMINAL
23,18,9,IC,973922,18.0,9.0,XCVT,INDUSTRY,18,9,...,NaN,0.0,30.052928,-90.831935,1,2,D,J,P,BATON ROUGE
24,18,9,UP,0918LV014,NaN,NaN,NaN,NaN,18,9,...,NaN,0.0,30.525485,-91.555759,2,5,K,NaN,NaN,LIVONIA SUB
25,18,8,UP,0818LV002,NaN,NaN,NaN,NaN,18,8,...,NaN,0.0,30.330582,-91.240827,2,5,K,NaN,NaN,LIVONIA SUB
26,18,8,UP,0818LV007,NaN,NaN,NaN,NaN,18,8,...,NaN,0.0,29.918621,-90.208067,2,5,K,NaN,NaN,LIVONIA SUB


### Operation 1 
Download new data from the FRA site ( ), import it into Pandas, grab only the new observations ( ), and append
these to .

In [9]:
df = pd.concat([df, df2])

In [10]:
df.shape

(205216, 145)

### Operation 2
Clean and append data

In [11]:
df = df2
df.shape

(386, 145)

### 1) Replacing numeric values of TYPE with their descriptive names.

In [12]:
df['TYPE'].value_counts()

1     240
12     47
7      31
9      22
13     19
4       9
3       7
11      6
5       3
6       2
Name: TYPE, dtype: int64

In [13]:
# converting TYPE column to categorical
df['TYPE'] = df['TYPE'].astype('category')

# assigning new categories
df['TYPE'].cat.categories = ['Derailment', 'Rear end collision', 'Side collision', 'Raking collision', 'Broken train collision', 'Hwy-rail crossing', 'Obstruction', 'Fire/violent rupture', 'Other impacts', 'Other']

### 2) Replacing numeric values of TYPEQ with their descriptive names.

In [14]:
# filling missing values with '0'
df['TYPEQ'] = df['TYPEQ'].fillna('0')

# treat 1.0 similar to 1 and 2.0 to 2 etc.
df['TYPEQ'] = df['TYPEQ'].apply(lambda x: x[0])

# converting to type categorical
df['TYPEQ'] = df['TYPEQ'].astype('category')
df['TYPEQ'].cat.categories = ['Missing', 'Freight Train', 'Passenger Train-Pulling', 'Commuter Train-Pulling', 'Work train', 'Single Car', 'Cut of cars', 'Yard/switching', 'Light loco(s)', 'Maint./inspect. Car', 'Spec. MoW Equip.', 'Commuter Train-Pushing', 'EMU']

### 3) Replacing CAUSE with first letter of code.

In [15]:
df['CAUSE'] = df['CAUSE'].apply(lambda x: x[0])
df.CAUSE.head()

21    E
23    M
24    M
25    T
26    H
Name: CAUSE, dtype: object

### 4) Imputing missing values in TYPEQ. 

In [16]:
# converting missing category back to NaN
df.loc[df['TYPEQ'] == 'Missing', 'TYPEQ'] = np.nan

# imputing missing values using mode since TYPEQ is categorical variable
df['TYPEQ'] = df['TYPEQ'].fillna(df['TYPEQ'].mode()[0])

### 5) Joining narratives.

In [17]:
# get all columns with NARR
narrative_columns = []
for i in range(1, 16):
    narrative_columns.append('NARR' + str(i))
    
# impute missing values by an empty string
df[narrative_columns] = df[narrative_columns].fillna('')
  
# concatenate all NARR columns into one single Narrative column
df['Narrative'] = ''
for column in narrative_columns:
    df['Narrative'] = df['Narrative'] + df[column]
    
# Dropping the older narrative columns  
df = df.drop(columns = narrative_columns)

In [18]:
df.head()

,IYR,IMO,RAILROAD,INCDTNO,IYR2,IMO2,RR2,INCDTNO2,IYR3,IMO3,...,RCL,Latitude,Longitud,SIGNAL,MOPERA,ADJUNCT1,ADJUNCT2,ADJUNCT3,SUBDIV,Narrative
21,18,8,KCS,18082502,NaN,NaN,NaN,NaN,18,8,...,0.0,32.541800,-93.844812,1,5,Z,NaN,NaN,SHREVEPORT TERMINAL,STARTED MOVEMENT IN THE NORTH TRACK & ENGINE D...
23,18,9,IC,973922,18.0,9.0,XCVT,INDUSTRY,18,9,...,0.0,30.052928,-90.831935,1,2,D,J,P,BATON ROUGE,CONVENT MARINE TERMINAL EMPLOYEES WERE IN THE ...
24,18,9,UP,0918LV014,NaN,NaN,NaN,NaN,18,9,...,0.0,30.525485,-91.555759,2,5,K,NaN,NaN,LIVONIA SUB,"LLM20R-21 FOUND CARS CRDX290017, CRDX290344 & ..."
25,18,8,UP,0818LV002,NaN,NaN,NaN,NaN,18,8,...,0.0,30.330582,-91.240827,2,5,K,NaN,NaN,LIVONIA SUB,THE LLL43B WAS EXITING THE GATE AT DOW AND THE...
26,18,8,UP,0818LV007,NaN,NaN,NaN,NaN,18,8,...,0.0,29.918621,-90.208067,2,5,K,NaN,NaN,LIVONIA SUB,THE YAV52-03 RAN THROUGH THE EAST END OF CROSS...


### 6) Dropping duplicates.

In [19]:
#Dropping duplicates based on JOINTCD
df = df[df['JOINTCD'] == 1]
df = df.drop_duplicates(keep='first')
df.shape

(324, 131)

### 7) Correcting for time value of money.

In [20]:
df_cpi = pd.read_csv('cpi.csv')
df_cpi.month = df_cpi.month
df_cpi.year = df_cpi.year
df_cpi['inflation'] = 252.885/df_cpi['cpi']
df_cpi.head()

,month,year,cpi,inflation
0,1,1975,52.1,4.853839
1,1,1976,55.6,4.548291
2,1,1977,58.5,4.322821
3,1,1978,62.5,4.046160
4,1,1979,68.3,3.702562


In [21]:
#Renaming columns
df_cpi.columns = ['MONTH', 'YEAR4', 'cpi', 'inflation']

In [22]:
merged_df = pd.merge(left = df, right = df_cpi, how = 'inner', left_on=['MONTH', 'YEAR4'], right_on = ['MONTH', 'YEAR4'])

In [23]:
merged_df.head()

,IYR,IMO,RAILROAD,INCDTNO,IYR2,IMO2,RR2,INCDTNO2,IYR3,IMO3,...,Longitud,SIGNAL,MOPERA,ADJUNCT1,ADJUNCT2,ADJUNCT3,SUBDIV,Narrative,cpi,inflation
0,18,8,KCS,18082502,NaN,NaN,NaN,NaN,18,8,...,-93.844812,1,5,Z,NaN,NaN,SHREVEPORT TERMINAL,STARTED MOVEMENT IN THE NORTH TRACK & ENGINE D...,252.146,1.002931
1,18,8,UP,0818LV002,NaN,NaN,NaN,NaN,18,8,...,-91.240827,2,5,K,NaN,NaN,LIVONIA SUB,THE LLL43B WAS EXITING THE GATE AT DOW AND THE...,252.146,1.002931
2,18,8,UP,0818LV007,NaN,NaN,NaN,NaN,18,8,...,-90.208067,2,5,K,NaN,NaN,LIVONIA SUB,THE YAV52-03 RAN THROUGH THE EAST END OF CROSS...,252.146,1.002931
3,18,8,BNSF,PR0818110,NaN,NaN,NaN,NaN,18,8,...,-104.624736,2,5,K,NaN,NaN,PIKES PEAK,RCO Y-PUE3022-30 HAD 3 RAILCARS DERAIL AND IMP...,252.146,1.002931
4,18,8,UP,0818DV013,NaN,NaN,NaN,NaN,18,8,...,-104.816089,1,1,Q,NaN,NaN,GREELEY SUB,"MCYDV-24, LEAD LOCOMOTIVE UP8902, STRUCK A TRA...",252.146,1.002931


In [24]:
merged_df['Adj_ACCDMG'] = merged_df['ACCDMG'] * merged_df['inflation']

In [25]:
merged_df[['ACCDMG', 'Adj_ACCDMG']].tail()

,ACCDMG,Adj_ACCDMG
319,1569161,1.571933e+06
320,43450,4.352677e+04
321,941187,9.428499e+05
322,12870,1.289274e+04
323,27600,2.764876e+04


#### Store in a table

In [26]:
merged_df.to_sql('df_clean', conn, if_exists='append', index=True)

In [27]:
conn.close()